In [12]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

### Dacon 제공 NA removal

In [ ]:
test = pd.read_csv("data/test.csv", sep = ",")
submission = pd.read_csv("data/submission.csv")

In [14]:
test.head()

,Time,X26,X303,X241,X435,X402,X352,X305,X350,X326,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
0,2017.7.1 0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN
1,2017.7.1 1:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222,0.021,NaN
2,2017.7.1 2:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237,0.021,NaN
3,2017.7.1 3:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229,0.020,NaN
4,2017.7.1 4:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202,0.021,NaN


In [17]:
test2 = test

In [26]:
counting[:-1]

Int64Index([1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460,
            ...
            8749, 8750, 8751, 8752, 8753, 8754, 8755, 8756, 8757, 8758],
           dtype='int64', length=6650)

In [27]:
# Time칼럼을 제외한 X26부터 마지막 칼럼까지 for loop
for k in range(1, len(test2.columns) ): 
    test_median=test2.iloc[:,k].median()   # 해당 칼럼(k)의 중앙값 median 계산
    counting=test2.loc[ test2.iloc[:,k].isnull()==False ][ test2.columns[k] ].index   # 해당칼럼에서 null=False(값이 존재)인 row index(시간) 추출

    df=DataFrame(list(zip(counting[:-1], #값이 존재하는 index(시간) 중 마지막 제외
                          counting[1:]-counting[:-1] -1)), # index 중 첫번째 제외 - 마지막 제외 - 1 : 각 인덱스 사이 구간에 얼마의 차이가 있는가, 즉 Null값이 몇개나 존재했는가
                 columns=['index','count'] )

    df2= df[ (df['count'] > 0) ]  # 직후에 Null이 1개 이상 존재하는 인덱스&null count만 뽑아냄
    df2=df2.reset_index(drop=True) 
    
    for i,j in zip( df2['index'], df2['count'] ) :  
        if test2.iloc[i,k]>=test_median:  # if 해당 인덱스에서 값이 median 이상일 경우
            test2.iloc[ i : i+j+1 , k] = test2.iloc[i,k] / (j+1)  # 해당 인덱스(시간) 포함 직후의 null값들에 해당 전력량/(count+1)만큼 분배한다
        else:  
            pass  # 아닐 경우 pass
        
    if k%50==0: #for문 진행정도 확인용
            print(k,"번째 실행중")

50 번째 실행중
100 번째 실행중
150 번째 실행중
200 번째 실행중


In [39]:
test2 = test2.set_index('Time')
test2.head()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017.7.1 0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN
2017.7.1 1:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222,0.021,NaN
2017.7.1 2:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237,0.021,NaN
2017.7.1 3:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229,0.020,NaN
2017.7.1 4:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202,0.021,NaN


In [40]:
test2.to_csv("data/dacon_naremoval_method_TEST.csv")

<br>

### 시간별 평균 대입
(코드 설명은 02_1 에 되어있음)

In [42]:
test = pd.read_csv("data/dacon_naremoval_method_TEST.csv", sep=',')

In [43]:
test.head()

,Time,X26,X303,X241,X435,X402,X352,X305,X350,X326,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
0,2017.7.1 0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN
1,2017.7.1 1:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222,0.021,NaN
2,2017.7.1 2:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237,0.021,NaN
3,2017.7.1 3:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229,0.020,NaN
4,2017.7.1 4:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202,0.021,NaN


In [44]:
test['Time'] = pd.to_datetime(test.Time)
test = test.set_index('Time')

In [45]:
test['month'] = test.index.month
test['date'] = test.index.day
test['hour'] = test.index.hour

In [46]:
test.head()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X266,X20,X443,X347,X75,X107,X230,month,date,hour
Time,,,,,,,,,,,,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.275,0.021,NaN,7,1,0
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.222,0.021,NaN,7,1,1
2017-07-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.237,0.021,NaN,7,1,2
2017-07-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.229,0.020,NaN,7,1,3
2017-07-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.202,0.021,NaN,7,1,4


In [47]:
hourly = test.groupby('hour').mean()
c = hourly.columns

test2 = pd.DataFrame()

n=0
for i in c:
    n +=1
    li = test[test[i].isnull()].index
    df = pd.DataFrame(test[i])
    
    print("%d. current column: %s, NaN count: %d" %(n, i, len(li)))
    for time in li:
        df.loc[time] = hourly[i][time.hour]
    test2 = pd.concat([test2,df], axis=1)   

1. current column: X26, NaN count: 7003
2. current column: X303, NaN count: 1426
3. current column: X241, NaN count: 1468
4. current column: X435, NaN count: 1547
5. current column: X402, NaN count: 1594
6. current column: X352, NaN count: 1575
7. current column: X305, NaN count: 1536
8. current column: X350, NaN count: 1579
9. current column: X326, NaN count: 1762
10. current column: X299, NaN count: 1654
11. current column: X160, NaN count: 1881
12. current column: X125, NaN count: 0
13. current column: X211, NaN count: 1532
14. current column: X349, NaN count: 1519
15. current column: X407, NaN count: 1554
16. current column: X330, NaN count: 1533
17. current column: X132, NaN count: 1509
18. current column: X231, NaN count: 1710
19. current column: X225, NaN count: 1712
20. current column: X16, NaN count: 7003
21. current column: X286, NaN count: 1663
22. current column: X354, NaN count: 1626
23. current column: X353, NaN count: 1574
24. current column: X76, NaN count: 0
25. curren

198. current column: X75, NaN count: 0
199. current column: X107, NaN count: 0
200. current column: X230, NaN count: 1895
201. current column: month, NaN count: 0
202. current column: date, NaN count: 0


In [48]:
test2.tail()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X223,X266,X20,X443,X347,X75,X107,X230,month,date
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-30 19:00:00,1.255,0.175,0.430,0.429,0.169,0.739,0.573,0.341,0.127,0.286,...,0.479,0.858,11.546,0.428,0.134,3.519,0.027,0.286,6,30
2018-06-30 20:00:00,1.596,1.692,0.332,0.366,0.159,0.255,0.501,0.281,0.126,0.230,...,0.725,0.816,11.591,0.413,0.206,3.331,0.027,0.303,6,30
2018-06-30 21:00:00,1.379,0.883,0.435,0.497,0.166,0.233,0.634,0.401,0.439,0.346,...,0.426,0.659,10.766,0.466,0.324,3.416,0.026,0.253,6,30
2018-06-30 22:00:00,1.000,0.539,0.268,0.459,0.463,0.454,0.676,0.543,0.737,0.466,...,0.353,0.462,0.312,0.473,0.402,4.263,0.027,0.253,6,30
2018-06-30 23:00:00,0.976,0.481,0.220,0.418,0.646,0.419,0.680,0.499,0.304,0.405,...,0.349,0.792,0.309,0.519,0.334,0.784,0.027,0.310,6,30


In [49]:
test3 = test2.drop(columns=['month', 'date'])

In [50]:
test3.to_csv("data/데이콘전처리및시간별평균.csv")

<br>

### Dacon 중급 베이스코드 (test셋의 각 미터ID마다 ARIMA 단순 시계열 예측)

In [55]:
submission = pd.read_csv("data/submission.csv")

In [51]:
time = []; place_id = []; target = []

for i in test3.columns:
    for j in range(len(test3)):
        place_id.append(i)
        time.append(test3.index[j])
        target.append(test3[i].iloc[j])
new_df = pd.DataFrame({'place_id': place_id, 'time':time, 'target': target})
new_df = new_df.dropna()
new_df = new_df.set_index('time')
new_df.head()

,place_id,target
time,,
2017-07-01 00:00:00,X26,1.187315
2017-07-01 01:00:00,X26,1.186123
2017-07-01 02:00:00,X26,1.168301
2017-07-01 03:00:00,X26,1.163425
2017-07-01 04:00:00,X26,1.398466


In [52]:
import itertools

In [53]:
p=d=q = range(0,2)
pdq = list(itertools.product(p,d,q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [54]:
def get_optimal_params(y):
    param_dict = {}
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [56]:
# agg라는 dictionary를 생성하여, 각 미터ID를 key로, 예측시간 및 값 리스트를 value로 저장
agg={}

for key in new_df['place_id'].unique(): # 미터ID리스트를 unique()함수로 추출, for loop
    temp = new_df.loc[new_df['place_id']==key] # new_df에서 key와 일치하는 place_id를 가지는 부분을 temp에 할당
    temp_1h=temp.resample('1h').sum() # 1시간 단위 resampling(일종의 timeseries압축)
    temp_1day=temp.resample('D').sum() # 1일 단위 resampling

    # 1 시간별 예측
    model = ARIMA(temp_1h['target'],  # target 을 추측하고자 함
                  order=get_optimal_params(temp_1h['target'])) # AIC를 최소화하는 최적의 파라미터 
    results_ARIMA = model.fit(disp=-1)  
    fcst = results_ARIMA.forecast(24) # 24시간 예측

    a = pd.DataFrame() # a: 예측값을 담을 데이터프레임 생성
    
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[0][i]] # column명 지정 및 예측값 대입

        
    # 1 일별 예측
    model = ARIMA(temp_1day['target'], 
                  order=get_optimal_params(temp_1day['target'])) 
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(10) # 10일을 예측

    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=[fcst[0][i]] # column명 지정 및 예측값 대입
    
    
    # 월별 예측
    # 일별 예측의 sum 사용
    fcst = results_ARIMA.forecast(153)
    
    # column명을 지정 및 예측값 대입
    a['X2018_7_m'] = [np.sum(fcst[0][:31])] # 7월 
    a['X2018_8_m'] = [np.sum(fcst[0][31:62])] # 8월
    a['X2018_9_m'] = [np.sum(fcst[0][62:92])] # 9월
    a['X2018_10_m'] = [np.sum(fcst[0][92:123])] # 10월
    a['X2018_11_m'] = [np.sum(fcst[0][123:153])] # 11월
    
    # a 에 meter_id를 현재 예측하고 있는 열의 id(key)로 대입
    a['meter_id'] = key 
    
    # agg{ 미터ID: 시간별(subimssion.columns는 예측시간칼럼들) a의 예측값}
    agg[key] = a[submission.columns.tolist()]
    print(key)
print('---- Modeling Done ----')

X26
X303


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X241
X435
X402
X352
X305
X350
X326
X299
X160
X125
X211
X349
X407
X330
X132
X231
X225
X16


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X286
X354
X353
X76


...

X368
X282
X426


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X47
X192
X283


c:\users\soomin\appdata\local\programs\python\python37-32\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X329
X223
X266
X20
X443
X347
X75
X107
X230
---- Modeling Done ----


In [57]:
output1 = pd.concat(agg, ignore_index = False)
output2 = output1.reset_index().drop(['level_0', 'level_1'], axis=1)
output2['id'] = output2['meter_id'].str.replace('X','').astype(int)
output2 = output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
output2.head()

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X5,6.853865,6.662422,6.535229,6.450733,6.394607,6.357335,6.332590,6.316171,6.305283,...,151.158683,151.165313,151.188370,151.219687,151.255157,4700.315581,4734.280878,4617.861169,4809.298872,4690.459229
1,X7,1.258811,1.275497,1.289773,1.301985,1.312433,1.321372,1.329018,1.335560,1.341157,...,40.379062,40.218088,40.145693,40.118419,40.114129,1268.281864,1265.220188,1242.315948,1302.232771,1278.134577
2,X10,2.166754,2.297002,2.408627,2.504290,2.586275,2.656538,2.716753,2.768359,2.812586,...,73.933546,73.933546,73.933546,73.933546,73.933546,2303.938641,2291.939926,2218.006380,2291.939926,2218.006380
3,X16,3.294517,3.381436,3.454943,3.517108,3.569681,3.614141,3.651741,3.683540,3.710432,...,112.487417,112.467445,112.489420,112.529645,112.577808,3514.788117,3552.022597,3487.105302,3654.661693,3586.433459
4,X18,2.500809,2.510078,2.517351,2.523060,2.527539,2.531054,2.533812,2.535976,2.537673,...,61.992220,61.938487,61.909200,61.893237,61.884536,1923.494138,1918.097489,1856.223376,1918.097488,1856.223376


In [58]:
output2.to_csv('result/02.dacon전처리및시간별평균.csv', index=False)